In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import numpy as np
import pandas as pd
import plotly.express as px
from investment import (
    SimulationConfig,
    Mortgage,
    Investment,
    ETFInvestment,
    LeveragedProperty
)

In [10]:
def run_fair_comparison(start_deposit, property_price, years):
    # 1. Init Config
    config = SimulationConfig(
        marginal_tax_rate=0.37,  # 37% Tax Bracket
        stamp_duty_rate=0.04     # 4% Stamp Duty
    )
    
    # 2. Calculate "Fair" Starting Capital
    # Property Investor pays: Deposit + Stamp Duty + Closing Costs
    stamp_duty = property_price * config.stamp_duty_rate
    entry_costs = stamp_duty + config.closing_costs
    
    total_cash_required = start_deposit + entry_costs
    
    loan_amount = property_price - start_deposit
    
    print(f"--- SETUP ---")
    print(f"Property Price: ${property_price:,.0f}")
    print(f"Loan Amount:    ${loan_amount:,.0f}")
    print(f"Upfront Cash:   ${total_cash_required:,.0f} (Deposit: ${start_deposit/1000}k + Fees: ${entry_costs/1000}k)")
    print(f"The ETF investor therefore starts with: ${total_cash_required:,.0f}")
    print("-" * 80)

    # 3. Setup Assets
    my_mortgage = Mortgage(principal=loan_amount, interest_rate=0.06, term_years=30)
    
    my_property = LeveragedProperty(
        "House", 
        purchase_price=property_price, 
        growth_rate=0.05, 
        rental_yield=0.04, 
        expenses=20_000, 
        mortgage=my_mortgage,
        config=config
    )
    
    my_etf = ETFInvestment(
        "S&P 500", 
        initial_cash=total_cash_required, # Fair Start!
        annual_return=0.08, 
        mer_fee=0.002, 
        config=config
    )

    print(f"{'Year':<5} | {'Morgage':<12} |{'Prop Cost':<12} | {'ETF Inject':<12} | {'Prop Equity':<15} | {'ETF Value':<15}")
    print("-" * 80)

    for year in range(1, years + 1):
        # A. Property: Calculate Return first to get asset growth
        my_property.calculate_annual_return()
        
        # B. Property: Apply Costs & Calculate Net "Out of Pocket"
        # This returns the POSITIVE amount the investor had to pay to keep the property
        out_of_pocket_cost = my_property.apply_costs()
        
        # C. ETF: Fairness Injection
        # If property cost the human $5k, the ETF human invests $5k too.
        if out_of_pocket_cost > 0:
            my_etf.invest_cash(out_of_pocket_cost)
            
        # D. ETF: Run Growth
        my_etf.calculate_annual_return()
        my_etf.apply_costs()

        # Output
        print(f"{year:<5} | {my_property.total_mortgage_payment:<12,.0f} |{out_of_pocket_cost:<12,.0f} | {out_of_pocket_cost:<12,.0f} | {my_property.equity:<15,.0f} | {my_etf.value:<15,.0f}")

# Run It
run_fair_comparison(start_deposit=200_000, property_price=1_000_000, years=30)

--- SETUP ---
Property Price: $1,000,000
Loan Amount:    $800,000
Upfront Cash:   $243,000 (Deposit: $200.0k + Fees: $43.0k)
The ETF investor therefore starts with: $243,000
--------------------------------------------------------------------------------
Year  | Morgage      |Prop Cost    | ETF Inject   | Prop Equity     | ETF Value      
--------------------------------------------------------------------------------
1     | 58,119       |26,877       | 26,877       | 260,119         | 290,884        
2     | 58,119       |26,168       | 26,168       | 323,345         | 341,732        
3     | 58,119       |25,418       | 25,418       | 389,840         | 395,729        
4     | 58,119       |24,625       | 24,625       | 459,774         | 453,074        
5     | 58,119       |23,786       | 23,786       | 533,324         | 513,979        
6     | 58,119       |22,900       | 22,900       | 610,680         | 578,670        
7     | 58,119       |21,964       | 21,964       | 692,039   